In [2]:
import numpy as np
import cv2
from matplotlib import pyplot as plt


cap = cv2.VideoCapture(0)
i=0


while(True):
    ret, frame = cap.read() #1 frame acquise à chaque iteration
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #conversion en n&b
    cv2.imshow('Capture_Video', gray) #affichage
    key = cv2.waitKey(1) #on évalue la touche pressée
    if key & 0xFF == ord('q'): #si appui sur 'q'
        break #sortie de la boucle while
    if key & 0xFF == ord('c'):
        cv2.imwrite('Frame_TP'+str(i)+'.jpg',frame) #enregistrement des frames
        i+=1

cap.release()
cv2.destroyAllWindows() 

In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import math
import glob
images = glob.glob('*.png')

# Arrays to store object points and image points from all the images.
coord_mm = [] # 3d point in real world space
coord_px = [] # 2d points in image plane.
f = 4
i1 = 320
i2 = 240

imnum=0
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret,corners = cv2.findChessboardCorners(gray, (7,7), None)

    k=0  
    for [[i,j]] in corners:
        i=int(i)
        j=int(j)
        x=20*(k//7)
        y=20*(k%7)
        coord_px.append([i-i1,j-i2])
        coord_mm.append([x,y,120*imnum])
       
        k+=1
    imnum+=1



A_p=[]
U1_p=[]
U1_tilde = []

for j in range(len(coord_px)):
    coord_px[j][0] = coord_px[j][0] 
    coord_px[j][1] = coord_px[j][1] 

       



print(i1)
for i in range(len(coord_mm)):
    U1_p.append([coord_px[i][0]])
    line=[]
    line.append((coord_px[i][1])*coord_mm[i][0])   
    line.append((coord_px[i][1])*coord_mm[i][1])
    line.append((coord_px[i][1])*coord_mm[i][2])
    line.append(coord_px[i][1])
    line.append((-coord_px[i][0])*coord_mm[i][0])
    line.append((-coord_px[i][0])*coord_mm[i][1])
    line.append((-coord_px[i][0])*coord_mm[i][2])
    A_p.append(line)

A=np.array(A_p)
U1=np.array(U1_p)

L=np.linalg.pinv(A).dot(U1)


o2c_abs=1/math.sqrt(L[4]**2+L[5]**2+L[6]**2)
beta=o2c_abs*math.sqrt(L[0]**2+L[1]**2+L[2]**2)
o2c=-o2c_abs
o1c=o2c*L[3]/beta
r11=L[0][0]*o2c/beta
r12=L[1]*o2c/beta
r13=L[2]*o2c/beta
r21=L[4]*o2c
r22=L[5]*o2c
r23=L[6]*o2c
R1=np.array([r11,r12,r13])
R2=np.array([r21,r22,r23])
R3=np.cross(R1.transpose(),R2.transpose())
r31=R3[0][0]
r32=R3[0][1]
r33=R3[0][2]


theta=-math.atan(r23/r33)
gamma=-math.atan(r12/r11)
omega=math.atan(r13/(-r23*math.sin(theta)+r33*math.cos(theta)))

B_p=[]
R_p=[]
for i in range(len(coord_mm)):
    B_p.append([coord_px[i][1],-(r21*coord_mm[i][0]+r22*coord_mm[i][1]+r23*coord_mm[i][2]+o2c)])
    R_p.append(-coord_px[i][1]*(r31*coord_mm[i][0]+r32*coord_mm[i][1]+r33*coord_mm[i][2]))

B=np.array(B_p)
R=np.array(R_p)

M=np.linalg.pinv(B).dot(R)

f2 = M[1]
f1=f2*beta
o3c = M[0]

Mint = np.array([[f1,0,i1,0],[0,f2,i2,0],[0,0,1,0]])
Mext = np.array([[r11,r12,r13,o1c],[r21,r22,r23,o2c],[r31,r32,r33,o3c],[0,0,0,1]])
Mpass = Mint.dot(Mext)


i=0
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    for k in range(49):
        dot_mm=coord_mm[i*49+k]
        dot_mm.append(1)
        print(dot_mm)
        dot_px=Mpass.dot(dot_mm)
        dot_px=dot_px/dot_px[2]
        img = cv2.circle(gray,(int(dot_px[0]),int(dot_px[1])),5,(255, 0, 0),2)
    cv2.imshow("image",gray)
    key = cv2.waitKey(10000)
    i+=1

cv2.destroyAllWindows()



320
L= [[ -1.19050908e-05]
 [ -1.18992275e-02]
 [  1.95803800e-03]
 [  6.46286392e-01]
 [ -1.32930222e-02]
 [ -2.49874252e-03]
 [ -5.18596208e-03]]
[-331.51613447 -685.17003069]
[[  1.21439018e+02  -6.10613489e+02  -1.97083586e+02  -7.55164755e+04]
 [ -5.37244813e+02  -1.54032093e+02  -4.62563311e+02  -3.22648916e+04]
 [  3.81256616e-01  -1.49350495e-01  -9.05302461e-01  -3.31516134e+02]]
[0, 0, 0, 1]
[0, 20, 0, 1]
[0, 40, 0, 1]
[0, 60, 0, 1]
[0, 80, 0, 1]
[0, 100, 0, 1]
[0, 120, 0, 1]
[20, 0, 0, 1]
[20, 20, 0, 1]
[20, 40, 0, 1]
[20, 60, 0, 1]
[20, 80, 0, 1]
[20, 100, 0, 1]
[20, 120, 0, 1]
[40, 0, 0, 1]
[40, 20, 0, 1]
[40, 40, 0, 1]
[40, 60, 0, 1]
[40, 80, 0, 1]
[40, 100, 0, 1]
[40, 120, 0, 1]
[60, 0, 0, 1]
[60, 20, 0, 1]
[60, 40, 0, 1]
[60, 60, 0, 1]
[60, 80, 0, 1]
[60, 100, 0, 1]
[60, 120, 0, 1]
[80, 0, 0, 1]
[80, 20, 0, 1]
[80, 40, 0, 1]
[80, 60, 0, 1]
[80, 80, 0, 1]
[80, 100, 0, 1]
[80, 120, 0, 1]
[100, 0, 0, 1]
[100, 20, 0, 1]
[100, 40, 0, 1]
[100, 60, 0, 1]
[100, 80, 0, 1]
[100, 